In [47]:
import re
import requests
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


def get_dict_ratings_anime(anime, ratings):
    
    df_ratings = pd.DataFrame(ratings)    

    dict_ratings = {
        'anime' : anime,
        'votes' : df_ratings['rate_votes'].sum(),
        'weight' : df_ratings['rate_weight'].sum(),
        'rate' : round(df_ratings['rate_weight'].sum() / df_ratings['rate_votes'].sum(), 2),
    }
    
    # For each rating
    for i in range(1,6):
        dict_ratings[str(i)] = df_ratings[df_ratings['rate_class'] == i]['rate_votes'].values[0]
        
    return dict_ratings

url = 'https://www.crunchyroll.com/sitemap'
request = requests.get(url)
soup = BeautifulSoup(request.content)

sitemap = soup.findAll('loc')

languages_symbols = ['en-gb', 'es', 'es-es', 'pt-br', 'pt-pt', 'fr', 'de', 'ar', 'it', 'ru']
languages_symbols_url = [ f'https://www.crunchyroll.com/{i}' for i in languages_symbols]

anime_links = []
for url in tqdm(sitemap):
    request = requests.get(url.text)
    soup = BeautifulSoup(request.content)
    
    for link in soup.findAll('loc'):
        link = link.text

        # Looks only urls with len greater than https://www.crunchyroll.com/
        if len(link) > 28:
            if link.startswith('https://www.crunchyroll.com/' and 'forumtopic' not in link):
                if not link in languages_symbols_url:
                    re_search_tabs = re.search(r'(https://www\.crunchyroll\.com/.*)/', link)
                    if not re_search_tabs:
                        anime_links.append(link)
           
                    
anime_links = list(set(anime_links))

# Write list to make contribute with people that only need that
with open("./data/animes_list_urls.txt","w+") as f:
    f.write('\n'.join(anime_links))

100%|██████████████████████████████████████████████████████████████████████████████████| 66/66 [07:31<00:00,  6.84s/it]


In [64]:
anime_ratings = []
for anime_link in tqdm(anime_links):
    request = requests.get(anime_link)
    soup = BeautifulSoup(request.content)
    rating = soup.find('ul', {'class' : 'rating-histogram'})
    
    anime = anime_link.split('/')[-1]
    
    ratings = []
    for rate in rating.findAll('li'):
        rate_class = rate.find('div', {'class' : 'left num strong'})
        rate_votes = rate.find(lambda tag: tag.name == 'div' and tag['class'] == ['left'])
        rate_votes = re.search(r'(\d+)', rate_votes.text)
        if rate_class:
            rate_class = int(rate_class.text)
            if rate_votes:
                rate_votes = int(.group(1))
            else : 
                rate_votes = 0
                
            ratings.append({'rate_class' : rate_class, 'rate_votes' : rate_votes,
                            'rate_weight' : rate_class * rate_votes})
            
    anime_ratings.append(get_dict_ratings_anime(anime, ratings))

  7%|█████▍                                                                          | 86/1253 [01:38<20:04,  1.03s/it]

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
pd.DataFrame(anime_ratings)